# Gate Set Pauli Noise Learning (PauliGST)

By Senrui Chen, 2025 May 14

Contents of this notebook: 
1) Start by specifying a gateset and reduced noise model (Nearest-neighbor 2-local on a ring)
2) Then provide a collections of experiments in the form of $(\mathcal C,P)$, where $\mathcal C$ is a Clifford gate sequence (within the gate set) and $P$ is a Pauli observable.
3) The algorithms return the degrees of freedom that is learned by the provided experiments, and compare it with the size of reduced learnable space.
4) Qiskit Simulations for learning and estimating the noises.

In [35]:
import numpy as np
from numpy.linalg import matrix_rank as rank
from itertools import product
from bidict import bidict # bi-directional dictionary
from numpy import sqrt, exp, mod,random
from qiskit.quantum_info import Clifford, Pauli
from qiskit import QuantumCircuit
from qiskit import __version__

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error, ReadoutError


print("qiskit ==",__version__)

qiskit == 0.24.1


**Reminder 1: We will use the "natural" ordering of Pauli instead of qiskit's ordering. By "IZ" we mean "Z" on the 2nd qubit.**

This is different from Qiskit convention, so we need to reserve the Pauli strings when needed.

**Reminder 2: $P$.evolve($C$) gives $C^\dagger PC$ (Heisenberg picture). For Schrodinger picture uses $P$.evolve($C$.adjoint()).**

In [36]:
### Help functions
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3). Empty set is excluded."
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

def pt(pauli):
    pattern = []
    for p in pauli:
        if p == 'I':
            pattern.append(0)
        else:
            pattern.append(1)
    return(pattern) 


## Step 1. Set up model

### Define gateset

By default, a length-$m$ gate sequence is given by $G_mU_m\cdots G_2U_2G_1U_1$, where $G_i$ is a layer of entangling Clifford gates while $U_i$ is a layer of single-qubit gates.

Each Clifford layer is specified by its stabilizer & destabilizer

In [37]:
n =  12 # system size
isOpenBoundary = False
assert mod(n,4)==0, "num of qubits not a multiple of 4"

ng = 2 # number of hard layers

if isOpenBoundary is True:
    nnlist = [(i,i+1) for i in range(n-1)] # nearest-neighbor on a line (open boundary)
else:
    nnlist = [(i,mod(i+1,n)) for i in range(n)] # nearest-neighbor on a ring

gate = {} # non-parallel CZ's on a ring
qc_list = []
cliff_list = []

#layer 1
qc = QuantumCircuit(n)
for i in range(n>>1):
    qc.cx(2*i, 2*i+1)
print("gate 1")
print(qc)
qc1 = qc
cliff1 = Clifford(qc)
qc_list.append(qc)
cliff_list.append(cliff1)
gate[('layer1',)] = cliff1

#layer 2
qc = QuantumCircuit(n)
for i in range(n>>1):
    if isOpenBoundary and 2*i+2 == n: continue # add this to ensure open boundary
    qc.cx(2*i+1, mod(2*i+2,n))
print("gate 2")
print(qc)
qc2 = qc
cliff2 = Clifford(qc)
qc_list.append(qc)
cliff_list.append(cliff2)
gate[('layer2',)] = cliff2

gate 1
           
 q_0: ──■──
      ┌─┴─┐
 q_1: ┤ X ├
      └───┘
 q_2: ──■──
      ┌─┴─┐
 q_3: ┤ X ├
      └───┘
 q_4: ──■──
      ┌─┴─┐
 q_5: ┤ X ├
      └───┘
 q_6: ──■──
      ┌─┴─┐
 q_7: ┤ X ├
      └───┘
 q_8: ──■──
      ┌─┴─┐
 q_9: ┤ X ├
      └───┘
q_10: ──■──
      ┌─┴─┐
q_11: ┤ X ├
      └───┘
gate 2
           ┌───┐
 q_0: ─────┤ X ├
           └─┬─┘
 q_1: ──■────┼──
      ┌─┴─┐  │  
 q_2: ┤ X ├──┼──
      └───┘  │  
 q_3: ──■────┼──
      ┌─┴─┐  │  
 q_4: ┤ X ├──┼──
      └───┘  │  
 q_5: ──■────┼──
      ┌─┴─┐  │  
 q_6: ┤ X ├──┼──
      └───┘  │  
 q_7: ──■────┼──
      ┌─┴─┐  │  
 q_8: ┤ X ├──┼──
      └───┘  │  
 q_9: ──■────┼──
      ┌─┴─┐  │  
q_10: ┤ X ├──┼──
      └───┘  │  
q_11: ───────■──
                


### Define reduced noise model

params[num_of_index] = parameter_label

In [38]:
### Specify locality constraints

# 2-local nearest neighbor
wtmax = 2
faclist = [(k,) for k in range(n)] + [(k,k+1) for k in range(n-1)] # need this to be hashable

if not isOpenBoundary:
    faclist.append((0,n-1))

print(faclist)

### no-crosstalk
# wtmax = 1 # maximum weight
# faclist = [(k,) for k in range(n)] # need this to be hashable
# faclist

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (0, 11)]


In [39]:
### Add noise parameters

param = bidict()
npar = 0
# state-preparation (SP)
for fac in faclist:
    param[npar] = ('S',fac)
    npar += 1
# measurement (M)
for fac in faclist:
    param[npar] = ('M',fac)
    npar += 1
nspam = npar
# Gate
for gindex in gate.keys():
    for fac in faclist:
        wt = len(fac)
        for P in list(product(['X','Y','Z'],repeat=wt)):
            param[npar] = (*gindex, fac, P)
            npar += 1

print("num of parameters:",npar)
# for key,value in param.items():
#     print(key,value)

num of parameters: 336


In [40]:
### define embedding map

def add_M_noise(eqs,P): # P is natural ordering
    pattern = pt(P)
    # for k in range(len(pattern)):
    #     if pattern[k] != 0:
    #         indx = param.inverse[('M', (k,))]
    #         eqs[indx] += 1
    for fac in faclist:
        flag = True
        for loc in fac:
            if pattern[loc] == 0: 
                flag = False
        if flag: eqs[param.inverse[('M', fac)]] += 1

def add_S_noise(eqs,P): # P is natural ordering
    # assert(('X' not in P) and ('Y' not in P))
    ### If we assume always choosing the correct input basis, can omit
    pattern = pt(P)
    # for k in range(len(pattern)):
    #     if pattern[k] != 0:
    #         indx = param.inverse[('S', (k,))]
    #         eqs[indx] += 1
    for fac in faclist:
        flag = True
        for loc in fac:
            if pattern[loc] == 0: 
                flag = False
        if flag: 
            eqs[param.inverse[('S', fac)]] += 1


def add_G_noise(eqs,g,P): # P is natural ordering
  
    if (g[0]=='layer1') or (g[0]=='layer2'):
        pattern = pt(P)  
        for fac in faclist:
            flag = True
            for loc in fac:
                if pattern[loc] == 0: 
                    flag = False
            if flag: 
                indx = param.inverse[(g[0],fac,tuple([P[i] for i in fac]))]
                eqs[indx] += 1

    else:
        pass


## Step 2. Import experimental designs and compute rank

In [41]:
#help functions
def getEq(C,Q):
    "Gate the row of the design matrix with gate sequence C and Pauli observable Q"

    one_eqs = np.zeros(npar,dtype=int)

    # M noise
    add_M_noise(one_eqs, Q)

    # Gate sequence noise
    for g in C[::-1]:
        if g[0]=='cx':
            cliff = gate[('cx',g[1])]
            # print(cliff)
            # print(Q)
            Q = Pauli(Q[::-1]).evolve(Clifford(cliff)).to_label()[-n:][::-1] #drop a possible pm sign
            add_G_noise(one_eqs,g,Q)
        elif g[0]=='ss':
            (i,j) = g[1]
            qc = QuantumCircuit(n)
            qc.s([i,j])
            cliff = Clifford(qc)
            Q = Pauli(Q[::-1]).evolve(Clifford(cliff)).to_label()[-n:][::-1] #drop a possible pm sign
        elif g[0]=='szsx':
            qc = QuantumCircuit(n)
            [qc.s(i) for i in range(n) if i % 2 == 0]
            [qc.sx(i) for i in range(n) if i % 2 == 1]
            cliff = Clifford(qc)
            Q = Pauli(Q[::-1]).evolve(Clifford(cliff)).to_label()[-n:][::-1] #drop a possible pm sign
        elif g[0]=='szsx2':
            qc = QuantumCircuit(n)
            [qc.s(i) for i in range(n) if i % 2 == 1]
            [qc.sx(i) for i in range(n) if i % 2 == 0]
            cliff = Clifford(qc)
            Q = Pauli(Q[::-1]).evolve(Clifford(cliff)).to_label()[-n:][::-1] #drop a possible pm sign
        elif g[0]=='hn':
            qc = QuantumCircuit(n)
            [qc.h(i) for i in range(n)]
            cliff = Clifford(qc)
            Q = Pauli(Q[::-1]).evolve(Clifford(cliff)).to_label()[-n:][::-1] #drop a possible pm sign
        elif g[0]=='layer1':
            Q = Pauli(Q[::-1]).evolve(cliff1).to_label()[-n:][::-1] #drop a possible pm sign
            add_G_noise(one_eqs,g,Q)
        elif g[0]=='layer2':
            Q = Pauli(Q[::-1]).evolve(cliff2).to_label()[-n:][::-1] #drop a possible pm sign
            add_G_noise(one_eqs,g,Q)
        elif g[0]=='sn':
            qc = QuantumCircuit(n)
            qc.s(list(range(n)))
            cliff = Clifford(qc)
            Q = Pauli(Q[::-1]).evolve(Clifford(cliff)).to_label()[-n:][::-1] #drop a possible pm sign
        else: assert 1==0, "gate undefined"
            
    # S noise
    add_S_noise(one_eqs, Q)

    return one_eqs

def interpret(eqs):
    "Print one row of design matrix in a readable way"
    for k,cnts in enumerate(eqs):
        if cnts!=0:
            print(cnts, param[k])

def getCircuit(C):
    "gate qiskit circuit from gate sequence"
    qc = QuantumCircuit(n)
    for g in C:
        if g[0] == 'cx':
            qc.cx(g[1][0],g[1][1])
        elif g[0] == 'ss':
            qc.s(g[1])
        elif g[0] == 'sx':
            qc.sx(g[1])
        elif g[0] == 'sn':
            qc.s(range(n))
        elif g[0] == 'szsx':
            [qc.s(i) for i in range(n) if i % 2 == 0]
            [qc.sx(i) for i in range(n) if i % 2 == 1]
        elif g[0] == 'szsx2':
            [qc.s(i) for i in range(n) if i % 2 == 1]
            [qc.sx(i) for i in range(n) if i % 2 == 0]
        elif g[0] == 'hn':
            [qc.h(i) for i in range(n)]
        elif g[0] == 'layer1':
            # qc = qc.compose(cliff1.to_circuit())
            # qc = cliff1.to_circuit()
            # qc.cz(0,1)
            # for i in range(n>>1):
            #     qc.cz(2*i, mod(2*i+1,n))
            qc = qc.compose(qc1)
        elif g[0] == 'layer2':
            # qc = qc.compose(cliff2.to_circuit())
            # qc.cz(1,2)
            # for i in range(n>>1):
            #     if isOpenBoundary:
            #         if 2*i+2 == n: continue
            #         qc.cz(2*i+1, mod(2*i+2,n))
            qc = qc.compose(qc2)
        else: assert 1==0, "gate undefined"
    return qc

def addExp(design_mat, exp_list, C,P):
    "One row update of the designe matrix"
    design_mat = np.vstack([design_mat, getEq(C,P)])
    exp_list.append((C,P))
    return design_mat, exp_list


# np.vstack([design_mat,experiment(1,2)])

### Simple design: only depth 0/1

In [42]:
design_mat = np.array(np.empty((0,npar),dtype=int))
exp_list = []

print("shape / rank / learnable DOFs")

#depth 0
#weight 1
for k in range(n):
    P0 = "Z" + "I"*(n-1)
    P = "".join([P0[mod(j-k,n)] for j in range(n)])
    design_mat,exp_list = addExp(design_mat,exp_list,[],P)

print(np.shape(design_mat), rank(design_mat), npar-n)

#weight 2
for k in range(n):
    P0 = "ZZ" + "I"*(n-2)
    P = "".join([P0[mod(j-k,n)] for j in range(n)])
    design_mat,exp_list = addExp(design_mat,exp_list,[],P)

print(np.shape(design_mat), rank(design_mat), npar-n)

#depth 1
#weight 1
for k in range(n):
    for P0 in ["X","Y","Z"]:
        P0 = P0 + "I"*(n-1)
        P = "".join([P0[mod(j-k,n)] for j in range(n)])
        design_mat,exp_list = addExp(design_mat,exp_list,[("layer1",)],P)
        design_mat,exp_list = addExp(design_mat,exp_list,[("layer2",)],P)
    
print(np.shape(design_mat), rank(design_mat), npar-n)

#weight 2
for k in range(n):
    for P0 in product(['X','Y','Z'],repeat=2):
        P0 = "".join(P0)
        P0 = P0 + 'I'*(n-2)
        P = "".join([P0[mod(j-k,n)] for j in range(n)])
        design_mat,exp_list = addExp(design_mat,exp_list,[("layer1",)],P)
        design_mat,exp_list = addExp(design_mat,exp_list,[("layer2",)],P)

print(np.shape(design_mat), rank(design_mat), npar-n)

#weight 2 (but not nearest-neighbor)
for k in range(n):
    P0 = "XIIZ"
    P0 = P0 + 'I'*(n-len(P0))
    P = "".join([P0[mod(j-k,n)] for j in range(n)])
    # print(P)
    if mod(k,2)==0:
        design_mat,exp_list = addExp(design_mat,exp_list,[("layer1",)],P)
    else:
        design_mat,exp_list = addExp(design_mat,exp_list,[("layer2",)],P)
        

print(np.shape(design_mat), rank(design_mat), npar-n)

shape / rank / learnable DOFs
(12, 336) 12 324
(24, 336) 24 324
(96, 336) 96 324


(312, 336) 312 324
(324, 336) 324 324


In [43]:
### print all depth-0/1 experiments
test_exp_list = []

for k in range(np.shape(design_mat)[0]):
    # print("---------")
    # print("Sequence:",exp_list[k])
    temp_circ = getCircuit(exp_list[k][0])
    temp_pauli = exp_list[k][1]
    temp_pauli_sp = Pauli(temp_pauli[::-1]).evolve(Clifford(temp_circ)).to_label()[-n:][::-1]
    # print(temp_circ)
    # print("Input:", temp_pauli_sp, "Measure:", exp_list[k][1])
    # interpret(design_mat[k,:])


    test_exp_list.append((temp_pauli_sp,exp_list[k][1],exp_list[k][0]))
    

### Design for amplifying gate noise

The following experiments does not add more DOF, but help amplify certain gate noise parameters.

In [44]:
### List of depth
# depth_list = [2,4,8] # please adjust to appropriate 
depth_list = [2]

In [45]:
design_mat2 = np.array(np.empty((0,npar),dtype=int))
exp_list2 = []

for depth in depth_list:
    for fac in faclist:
        wt = len(fac)
        for Ploc in product(['X','Y','Z'],repeat=wt):
            P = []
            for _j in range(n):
                if _j not in fac:
                    P.append("I")
                else:
                    P.append(Ploc[fac.index(_j)])
            P = "".join(P)
            # print(P)

            design_mat2,exp_list2 = addExp(design_mat2,exp_list2,[("layer1",)]*depth,P)
            design_mat2,exp_list2 = addExp(design_mat2,exp_list2,[("layer2",)]*depth,P)
            design_mat2,exp_list2 = addExp(design_mat2,exp_list2,[('szsx',),("layer1",)]*depth,P)
            design_mat2,exp_list2 = addExp(design_mat2,exp_list2,[('szsx2',),("layer2",)]*depth,P)



### weight3 & relational terms
### optional
for depth in depth_list:
    for k in range(n):
        P0 = "YIY" 
        P0 = P0 + 'I'*(n-len(P0))
        P = "".join([P0[mod(j-k,n)] for j in range(n)])
        design_mat2,exp_list2 = addExp(design_mat2,exp_list2,[('hn',),("layer1",),('hn',),("layer2",)]*depth,P) # CNOT layers do not commute. Add Hadamard to form cycles.



print(np.shape(design_mat2), rank(design_mat2), npar-n)


(588, 336) 299 324


In [46]:
# pcolormesh(design_mat2[:,:])


In [47]:
### print all amplified experiments

test_exp_rel = []

for k in range(np.shape(design_mat2)[0]):
    # print("---------")
    # print("Sequence:",exp_list2[k])
    temp_circ = getCircuit(exp_list2[k][0])
    temp_pauli = exp_list2[k][1]
    temp_pauli_sp = Pauli(temp_pauli[::-1]).evolve(Clifford(temp_circ)).to_label()[-n:][::-1]
    # print(temp_circ)
    # print("Input:", temp_pauli_sp, "Measure:", exp_list2[k][1])
    # interpret(design_mat2[k,:])

    test_exp_rel.append((temp_pauli_sp,exp_list2[k][1],exp_list2[k][0]))


In [48]:
### verify gate learnable
print("nspam =",nspam)
design_mat_rel = np.array(np.empty((0,npar-nspam),dtype=int))
for k in range(len(exp_list2)):
    eqs = design_mat2[k,:]
    not_rel = False
    for fac in faclist:
        if (eqs[param.inverse[('S',fac)]]!=eqs[param.inverse[('M',fac)]]):
            not_rel = True
            break
    if not_rel is True: continue
    design_mat_rel = np.vstack([design_mat_rel,eqs[nspam:]])

print(np.shape(design_mat_rel), rank(design_mat_rel), npar - nspam - n)

nspam = 48
(588, 288) 276 276


## Step 3. Parallel design

Compile design matrix into qiskit circuits


In [49]:
### help functions
def prepare_pauli_eigenstate_1q(circuit,index,pauli=None):
	# if pauli == 'I':
	# 	pauli = random.choice(['X','Y','Z']) 
	# note: For Pauli I, prepare an arbitrary eigenstate.
	if pauli == 'Z' or pauli == 'I':
		circuit.id([index])
	elif pauli == 'X':
		circuit.h([index])
	elif pauli == 'Y':
		circuit.h([index])
		circuit.s([index])
	else:
		assert 1==0

def pauli_gate_1q(circuit,index,pauli=None): #For stabilizer simulator to work, cannot use Pauli class
	if pauli == 'I':
		circuit.id([index])
	elif pauli == 'Z':
		circuit.z([index])
	elif pauli == 'X':
		circuit.x([index])
	elif pauli == 'Y':
		circuit.y([index])
	else:
		assert 1==0

def measure_pauli_1q(circuit,index,pauli=None):
	if pauli == 'I' or pauli == 'Z':
		circuit.id([index])
	elif pauli == 'X':
		circuit.h([index])
	elif pauli == 'Y':
		circuit.s([index])
		circuit.s([index])
		circuit.s([index])
		circuit.h([index])
	else:
		assert 1==0

def my_evolve(Q,cliff,signed=False): # natural ordering instead of qiskit ordering
	if not signed:
		return Pauli(Q[::-1]).evolve(Clifford(cliff)).to_label()[-n:][::-1] #drop a possible pm sign
	else:
		return Pauli(Q[::-1]).evolve(Clifford(cliff)).to_label()[::-1]

def add_circ(Q,P,qc_gate):
	qc = QuantumCircuit(n)
	for j in range(n):
		prepare_pauli_eigenstate_1q(qc,j,Q[j])
	qc.barrier()
	qc = qc.compose(qc_gate)
	qc.barrier()
	for j in range(n):
		measure_pauli_1q(qc,j,P[j])
	# print(Q,P,k)
	# print(qc)
	qc.measure_all()
	return qc

In [50]:
circ_list = []
raw_exp_list = []

#depth 0
qc=QuantumCircuit(n)
qc.measure_all()
qc.draw()
circ_list.append(qc)
raw_exp_list.append(("Z"*n,"Z"*n,[]))

#depth 1
for k in range(2):
    # print(k)
    cliff = cliff_list[k]
    qck = qc_list[k]
    label = [("layer1",),("layer2",)][k]
    # print(label)

    #aligned
    for P in product(["X","Y","Z"],repeat=2):
        if P == ("I","I"): continue
        P = "".join(P)*(n>>1) # measure P
        Q = my_evolve(P,cliff) # Input Q
        qc = add_circ(Q,P,qck)
        circ_list.append(qc)
        raw_exp_list.append((Q,P,[label]))
        
    #between
    for P0 in product(["X","Y","Z"],repeat=2):
        P0 = ("I"+"".join(P0)+"I") * (n>>2) # measure P
        for shift in [k,k+2]:
            P = "".join([P0[mod(j-shift,n)] for j in range(n)])
            Q = my_evolve(P,cliff) # Input Q
            # print(Q,P)
            qc = add_circ(Q,P,qck)
            circ_list.append(qc)
            raw_exp_list.append((Q,P,[label]))
        
len(raw_exp_list) #, raw_exp_list

55

In [51]:
#help functions
def majby(a,b):
    "Return True if a is majorized by b."
    assert len(a)==len(b)
    flag = True
    for k in range(len(a)):
        if a[k] != "I" and b[k] != a[k]:
            flag = False
            break
    return flag

In [52]:
### Validate completeness of parllel circuits
nfl = []
from_circ = [] # from which raw exp? (index)

for (Q,P,circ) in test_exp_list:
    # print(Q,P,circ)
    found = False
    for (ci, (Q1,P1,circ1)) in enumerate(raw_exp_list):
        if circ==circ1 and majby(Q,Q1) and majby(P,P1):
            found = True
            from_circ.append(ci)
            # print("from", Q1,P1,circ1)
            break
    if not found: 
        # print("not found")
        from_circ.append(-1)
        nfl.append((Q,P,circ))

# for expexp in nfl: print(expexp)
print("Not found cnt:",len(nfl))

Not found cnt: 0


In [53]:
# Throw away circuits that are not used
select = [False] * len(raw_exp_list)
for ind in from_circ:
    select[ind] = True

raw_exp_opt = []
circ_opt = []
for ind in range(len(raw_exp_list)):
    if select[ind]:
        raw_exp_opt.append(raw_exp_list[ind])
        circ_opt.append(circ_list[ind])

len(raw_exp_list), len(raw_exp_opt)

(55, 51)

### Alternative design: relative precision estimation

I set gate depth as 2 for simplicity. In practice it can be a sequence looks like [2,4,8,16,32,...]

In [54]:
circ_rel = []
raw_exp_rel = []

# 1 gate
for depth in depth_list:
    for k in range(2):
        # print(k)
        cliff = cliff_list[k]
        qck = qc_list[k]
        label = [("layer1",),("layer2",)][k]

        #align
        for P in product(["X","Y","Z"],repeat=2):
            P = "".join(P)*(n>>1) # measure P, input P, cycle

            seq = [label]*depth
            qc = add_circ(P,P,getCircuit(seq))
            circ_rel.append(qc)
            raw_exp_rel.append((P,P,seq))
            
            if k == 0:
                seq = [("szsx",),label]*depth
            else:
                seq = [("szsx2",),label]*depth
            qc = add_circ(P,P,getCircuit(seq))
            circ_rel.append(qc)
            raw_exp_rel.append((P,P,seq))


# 2 gate 
# optional
for depth in depth_list:
    P = "YI" * (n>>1)
    # seq = [("layer1",),("layer2",)]*2
    seq = [("hn",),("layer1",),("hn",),("layer2",)]*depth
    qc = add_circ(P,P,getCircuit(seq))
    circ_rel.append(qc)
    raw_exp_rel.append((P,P,seq))

    P = "IY" * (n>>1)
    seq = [("hn",),("layer1",),("hn",),("layer2",)]*depth
    qc = add_circ(P,P,getCircuit(seq))
    circ_rel.append(qc)
    raw_exp_rel.append((P,P,seq))

len(raw_exp_rel)

38

In [55]:
# for circ in circ_rel: print(circ)

#Validate completeness of parllel circuits
nfl_rel = []
from_circ_rel = [] # from which raw exp? (index)

for (Q,P,circ) in test_exp_rel:
    # print(Q,P,circ)
    found = False
    for (ci, (Q1,P1,circ1)) in enumerate(raw_exp_rel):
        if circ==circ1 and majby(Q,Q1) and majby(P,P1):
            found = True
            from_circ_rel.append(ci)
            # print("from", Q1,P1,circ1)
            break
    if not found: 
        print(Q,P,circ,"not found")
        from_circ_rel.append(-1)
        nfl_rel.append((Q,P,circ))

# for expexp in nfl: print(expexp)
print("Not found cnt:",len(nfl_rel))


Not found cnt: 0


In [56]:
# Throw away circuits that are not used
select = [False] * len(raw_exp_rel)
for ind in from_circ_rel:
    select[ind] = True

raw_exp_rel_opt = []
circ_rel_opt = []
for ind in range(len(raw_exp_rel)):
    if select[ind]:
        raw_exp_rel_opt.append(raw_exp_rel[ind])
        circ_rel_opt.append(circ_rel[ind])

len(raw_exp_rel), len(raw_exp_rel_opt)

(38, 38)

### Output

In [57]:
### print design for depth-0/1 circuits
# print("Depth 0/1 exps. cnt=",len(raw_exp_list))
# for item in raw_exp_list: print(item)

In [58]:
### print design for amplified circuits
# print("Depth for rel-prec exps. cnt=",len(raw_exp_rel))
# for item in raw_exp_rel: print(item)

In [59]:
### print an example of circuit construction

k = 1 # anyone 
print(raw_exp_rel[k]) #instruction
Q,P,seq = raw_exp_rel[k]  
gate_seq = getCircuit(seq) #from instruction to QuantumCircuit
qc = add_circ(Q,P,gate_seq)
qc.draw()


('XXXXXXXXXXXX', 'XXXXXXXXXXXX', [('szsx',), ('layer1',), ('szsx',), ('layer1',)])


┌───┐ ░ ┌───┐      ┌───┐       ░ ┌───┐ ░ ┌─┐                        »
    q_0: ┤ H ├─░─┤ S ├───■──┤ S ├───■───░─┤ H ├─░─┤M├────────────────────────»
         ├───┤ ░ ├───┴┐┌─┴─┐├───┴┐┌─┴─┐ ░ ├───┤ ░ └╥┘┌─┐                     »
    q_1: ┤ H ├─░─┤ √X ├┤ X ├┤ √X ├┤ X ├─░─┤ H ├─░──╫─┤M├─────────────────────»
         ├───┤ ░ ├───┬┘└───┘├───┬┘└───┘ ░ ├───┤ ░  ║ └╥┘┌─┐                  »
    q_2: ┤ H ├─░─┤ S ├───■──┤ S ├───■───░─┤ H ├─░──╫──╫─┤M├──────────────────»
         ├───┤ ░ ├───┴┐┌─┴─┐├───┴┐┌─┴─┐ ░ ├───┤ ░  ║  ║ └╥┘┌─┐               »
    q_3: ┤ H ├─░─┤ √X ├┤ X ├┤ √X ├┤ X ├─░─┤ H ├─░──╫──╫──╫─┤M├───────────────»
         ├───┤ ░ ├───┬┘└───┘├───┬┘└───┘ ░ ├───┤ ░  ║  ║  ║ └╥┘┌─┐            »
    q_4: ┤ H ├─░─┤ S ├───■──┤ S ├───■───░─┤ H ├─░──╫──╫──╫──╫─┤M├────────────»
         ├───┤ ░ ├───┴┐┌─┴─┐├───┴┐┌─┴─┐ ░ ├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐         »
    q_5: ┤ H ├─░─┤ √X ├┤ X ├┤ √X ├┤ X ├─░─┤ H ├─░──╫──╫──╫──╫──╫─┤M├─────────»
         ├───┤ ░ ├───┬┘└───┘├───┬┘└───┘ ░ ├───┤ ░  ║  ║  ║  ║  ║ └╥┘┌─┐      »
    q_6: ┤ H ├─░─┤ S ├───■──┤ S ├───■───░─┤ H ├─░──╫──╫──╫──╫──╫──╫─┤M├──────»
         ├───┤ ░ ├───┴┐┌─┴─┐├───┴┐┌─┴─┐ ░ ├───┤ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   »
    q_7: ┤ H ├─░─┤ √X ├┤ X ├┤ √X ├┤ X ├─░─┤ H ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├───»
         ├───┤ ░ ├───┬┘└───┘├───┬┘└───┘ ░ ├───┤ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐»
    q_8: ┤ H ├─░─┤ S ├───■──┤ S ├───■───░─┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├»
         ├───┤ ░ ├───┴┐┌─┴─┐├───┴┐┌─┴─┐ ░ ├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘»
    q_9: ┤ H ├─░─┤ √X ├┤ X ├┤ √X ├┤ X ├─░─┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
         ├───┤ ░ ├───┬┘└───┘├───┬┘└───┘ ░ ├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
   q_10: ┤ H ├─░─┤ S ├───■──┤ S ├───■───░─┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
         ├───┤ ░ ├───┴┐┌─┴─┐├───┴┐┌─┴─┐ ░ ├───┤ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
   q_11: ┤ H ├─░─┤ √X ├┤ X ├┤ √X ├┤ X ├─░─┤ H ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
         └───┘ ░ └────┘└───┘└────┘└───┘ ░ └───┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
meas: 12/══════════════════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩═»
                                                   0  1  2  3  4  5  6  7  8 »
«                  
«    q_0: ─────────
«                  
«    q_1: ─────────
«                  
«    q_2: ─────────
«                  
«    q_3: ─────────
«                  
«    q_4: ─────────
«                  
«    q_5: ─────────
«                  
«    q_6: ─────────
«                  
«    q_7: ─────────
«                  
«    q_8: ─────────
«         ┌─┐      
«    q_9: ┤M├──────
«         └╥┘┌─┐   
«   q_10: ─╫─┤M├───
«          ║ └╥┘┌─┐
«   q_11: ─╫──╫─┤M├
«          ║  ║ └╥┘
«meas: 12/═╩══╩══╩═
«          9  10 11

## Step 4. Simulations

The above codes generate experiments that are sufficient for data acquisition. Now we finish the codes for data processing. We also include qiskit simulation.

We reconstruct the noise parameters based on $A\bm{r}=\bm{b}$, where $\bm r$ is all noise parameters, $A$ is the design matrix, $\bm{b}$ is the observed (negative log) circuit fidelity. 

The data processing code first estimates $\hat{\bm b}$ from parallelized experiments. Then estimate $\hat{\bm r}$ using e.g. $\hat{\bm r} =A^+ \hat{\bm b}$. Note that $A$ is not full-rank due to gauge issue, but has maximum rank by design. In other words, $\hat{\bm r}$ has exactly $n$ single-qubit depolarizing gauge degrees of freedom, consistent with the theory.


In [60]:


# Simulation paramaters
shots = 10000
eps = 0.01
eps_readout = 0.01 
noise_model = NoiseModel()

# gate error (try a simple bitflip)
bit_flip_1q = pauli_error([('X', eps), ('I', 1 - eps)])
noise_model.add_all_qubit_quantum_error(bit_flip_1q.tensor(bit_flip_1q),['cx'])

# readout error
p0given1 = eps_readout
p1given0 = eps_readout
readout_noise_1q = ReadoutError([[1 - p1given0, p1given0], [p0given1, 1 - p0given1]])
noise_model.add_all_qubit_readout_error(readout_noise_1q)


backend = AerSimulator(method='stabilizer', noise_model=noise_model)
job = backend.run(circ_list, shots=shots, max_parallel_experiments=0, memory = True) 
result = job.result()

In [61]:
### using depth 0/1 experiments as an example
# raw_exp_results = []

from scipy.linalg import pinv

def estimate_from_raw(rawidx,obs):
    # Note 1: we have ignored any possible sign. Take some "absolute value" here.
    # Note 2: For gate amplyfing experiments. We might want to use multiple depth to fit an estimate.
    cnts = result.get_counts()[rawidx]
    res = 0
    tot = 0

    ptrev = pt(obs)[::-1] # to fit qiskit's convention
    
    for outcome,value in cnts.items():
        sgn = 1
        for k in range(len(outcome)):
            # print(ptrev,outcome)
            if ptrev[k] == 1 and outcome[k] == "1":
                sgn = -sgn
        res += sgn * value
        tot += value
    return np.abs(res/tot) # assume all Pauli fidelity >> 0

b = []

for k in range(len(exp_list)):
    obs = exp_list[k][1]
    rawidx = from_circ[k]
    # print("Estimate:",obs,"from raw exp:",raw_exp_list[rawidx]) # Estimate observable "obs" from raw experiments #rawidx
    b.append(estimate_from_raw(rawidx,obs)) 

b = np.array(b)
# print(b)

logb = -np.log(b) # minus log

In [62]:
A_p = pinv(design_mat)
r_0 = A_p @ logb # One solution
# x_0

Let $x_a^G = -\log\lambda_a^G$ for any Pauli $a$, the later is the familiar Pauli fidelity. Our noise model is given by $$x_a^G = \sum_{b\triangleleft a,~b~\text{is a factor}}r_b^G.$$
In our case the factor are all weight-1 Pauli and nearest-neighbor weight 2-Pauli. SPAM noise is given similarly, but with the Pauli eigenvalues only depending on their pattern (i.e., symmetrized over X, Y, Z). 

The following get_spam and get_gate function goes from $r$ to $x$.

In [63]:
#help function - get x from r
def get_spam(r, label, indset):
    x_tmp = 0.0
    for fac in powerset(indset):
        if fac in faclist: 
            x_tmp += r[param.inverse[(label, fac)]]
    return x_tmp

def get_gate(r, label, P):
    x_tmp = 0.0
    pattern = pt(P)
    indset = []
    for k in range(n):
        if pattern[k] == 1: 
            indset.append(k)
    # print(pattern,indset)
    for fac in powerset(indset):
        if fac in faclist: 
            # print()
            x_tmp += r[param.inverse[(label, fac, tuple([P[k] for k in fac]))]]
    return x_tmp
        

### Verify estimation

For SPAM noise, we simplify verify the product of SP and M eigenvalues.

In [64]:
#verify learnable fidelity... SPAM first
print("True 1q SPAM fidelity:", 1-2*eps_readout)
print("Estimated 2q SPAM (qubit 0,1,...,11):")
ave_err = 0.0
for k in range(n):
    # x_spam = x_0[param.inverse[('S', (k,))]] + x_0[param.inverse[('M', (k,))]]
    x_spam = get_spam(r_0,'S',(k,)) + get_spam(r_0,'M',(k,))
    print((k,),np.exp(-x_spam))
    ave_err += np.abs(np.exp(-x_spam) - (1-2*eps_readout))
ave_err/=n
print("average estimation error =",ave_err)

True 1q SPAM fidelity: 0.98
Estimated 2q SPAM (qubit 0,1,...,11):
(0,) 0.9813999999999999
(1,) 0.9753999999999997
(2,) 0.9794000000000005
(3,) 0.9793999999999999
(4,) 0.9817999999999997
(5,) 0.9819999999999997
(6,) 0.9804
(7,) 0.9807999999999997
(8,) 0.9774000000000002
(9,) 0.9836
(10,) 0.9806
(11,) 0.9800000000000001
average estimation error = 0.0015833333333332422


In [65]:
#verify learnable fidelity... SPAM first
print("True 2q SPAM fidelity:", (1-2*eps_readout)**2)
print("Estimated 2q SPAM fidelity (qubit (0,1),...,(11,0)):")
ave_err = 0.0
for k in range(n):
    x_spam = get_spam(r_0, 'S', (k,np.mod(k+1,n))) + get_spam(r_0, 'M', (k,np.mod(k+1,n))) 
    print((k,np.mod(k+1,n)),np.exp(-x_spam))
    ave_err += np.abs((1-2*eps_readout)**2 - np.exp(-x_spam))
ave_err/=n
print("average estimation error =",ave_err)

True 2q SPAM fidelity: 0.9603999999999999
Estimated 2q SPAM fidelity (qubit (0,1),...,(11,0)):
(0, 1) 0.9580000000000002
(1, 2) 0.9552000000000002
(2, 3) 0.9592000000000002
(3, 4) 0.9616
(4, 5) 0.9646
(5, 6) 0.9627999999999999
(6, 7) 0.9620000000000002
(7, 8) 0.9586
(8, 9) 0.9613999999999999
(9, 10) 0.9641999999999998
(10, 11) 0.9614000000000003
(11, 0) 0.961772
average estimation error = 0.0022643333333333218


In [66]:
#verify learnable fidelity... SPAM first
print("True 3q SPAM fidelity:", (1-2*eps_readout)**3)
print("Estimated 3q SPAM fidelity:")
ave_err = 0.0
for k in range(n):
    x_spam = get_spam(r_0, 'S', (k,np.mod(k+1,n),np.mod(k+2,n))) + get_spam(r_0, 'M', (k,np.mod(k+1,n),np.mod(k+2,n))) 
    print((k,np.mod(k+1,n),np.mod(k+2,n)),np.exp(-x_spam))
    ave_err += np.abs((1-2*eps_readout)**3 - np.exp(-x_spam))
ave_err/=n
print("average estimation error =",ave_err)

True 3q SPAM fidelity: 0.9411919999999999
Estimated 3q SPAM fidelity:
(0, 1, 2) 0.9381603444740626
(1, 2, 3) 0.9354991219113742
(2, 3, 4) 0.9417671227282012
(3, 4, 5) 0.9447538806274193
(4, 5, 6) 0.945740203665988
(5, 6, 7) 0.9447303141574867
(6, 7, 8) 0.9402255301794459
(7, 8, 9) 0.9429077552690811
(8, 9, 10) 0.9424378609190726
(9, 10, 11) 0.9453211095247808
(10, 11, 0) 0.9435179600000001
(11, 0, 1) 0.9388400000000003
average estimation error = 0.0028069341939289116


Now we verify gate noise eigenvalues estimation. Note the difference between learnable and unlearnable parameters.

In [67]:
#verify learnable gate fidelity
print("True 1q Gate Z fidelity (layer 1):", (1-2*eps))
ave_err = 0.0
for k in range(n>>1):
    P = "Z"+"I"*(n-1)
    P = "".join([P[mod(j-2*k,n)] for j in range(n)])
    x_gate = get_gate(r_0, 'layer1', P)  
    print(P,np.exp(-x_gate))
    ave_err += np.abs((1-2*eps) - np.exp(-x_gate))
ave_err/=n/2
print("average estimation error =",ave_err)

print("Below are gauge-dependent (unlearnable) fidelity")
ave_err = 0.0
for k in range(n>>1):
    P = "Z"+"I"*(n-1)
    P = "".join([P[mod(j-2*k-1,n)] for j in range(n)])
    x_gate = get_gate(r_0, 'layer1', P)  
    print(P,np.exp(-x_gate))
    # ["learnable","degenerate"][np.mod(k,2)]
    ave_err += np.abs((1-2*eps) - np.exp(-x_gate))
ave_err/=n/2
print("average estimation error =",ave_err,"(Large error due to unlearnability)")


print("Learnable combined cycles (IZ->ZZ). True value =",(1-2*eps) * (1-2*eps)**2)
ave_err = 0.0
for k in range(n>>1):
    P = "Z"+"I"*(n-1)
    P1 = "".join([P[mod(j-2*k-1,n)] for j in range(n)])
    P = "ZZ"+"I"*(n-2)
    P2 = "".join([P[mod(j-2*k,n)] for j in range(n)])
    x_gate = get_gate(r_0, 'layer1', P1) +  get_gate(r_0, 'layer1', P2) 
    print(P1,P2,np.exp(-x_gate))
    ave_err += np.abs((1-2*eps) * (1-2*eps)**2 - np.exp(-x_gate))
ave_err/=n/2
print("average estimation error =",ave_err)

True 1q Gate Z fidelity (layer 1): 0.98
ZIIIIIIIIIII 0.9818626452007337
IIZIIIIIIIII 0.9785583010006124
IIIIZIIIIIII 0.9755551028722758
IIIIIIZIIIII 0.9824561403508771
IIIIIIIIZIII 0.978514426028238
IIIIIIIIIIZI 0.9775647562716703
average estimation error = 0.0023543665631357382
Below are gauge-dependent (unlearnable) fidelity
IZIIIIIIIIII 0.9807090740583638
IIIZIIIIIIII 0.9736497072708179
IIIIIZIIIIII 0.9674693583017596
IIIIIIIZIIII 0.971403396757425
IIIIIIIIIZII 0.9721315108042626
IIIIIIIIIIIZ 0.9762325960457117
average estimation error = 0.006637084146397816 (Large error due to unlearnability)
Learnable combined cycles (IZ->ZZ). True value = 0.9411919999999999
IZIIIIIIIIII ZZIIIIIIIIII 0.9528495776905191
IIIZIIIIIIII IIZZIIIIIIII 0.9434910447972179
IIIIIZIIIIII IIIIZZIIIIII 0.9401731266466316
IIIIIIIZIIII IIIIIIZZIIII 0.9410518122589903
IIIIIIIIIZII IIIIIIIIZZII 0.9380306339550064
IIIIIIIIIIIZ IIIIIIIIIIZZ 0.9406844610113653
average estimation error = 0.003130764769290565


In [68]:
#verify learnable gate fidelity
print("True 1q Gate Z fidelity (layer 2):", (1-2*eps))
# print("Estimated 2q SPAM fidelity (qubit (0,1),...,(11,0)):")
ave_err = 0.0
for k in range(n>>1):
    P = "Z"+"I"*(n-1)
    P = "".join([P[mod(j-2*k-1,n)] for j in range(n)])
    x_gate = get_gate(r_0, 'layer2', P)  
    print(P,np.exp(-x_gate))
    ave_err += np.abs((1-2*eps) - np.exp(-x_gate))
ave_err/=n/2
print("average estimation error =",ave_err)

print("Below are gauge-dependent (unlearnable) fidelity")
ave_err = 0.0
for k in range(n>>1):
    P = "Z"+"I"*(n-1)
    P = "".join([P[mod(j-2*k,n)] for j in range(n)])
    x_gate = get_gate(r_0, 'layer2', P)  
    print(P,np.exp(-x_gate))
    ave_err += np.abs((1-2*eps) - np.exp(-x_gate))
ave_err/=n/2
print("average estimation error =",ave_err,"(Large error due to unlearnability)")


print("Learnable combined cycles (IZ->ZZ). True value =",(1-2*eps) * (1-2*eps)**2)
ave_err = 0.0
for k in range(n>>1):
    P = "Z"+"I"*(n-1)
    P1 = "".join([P[mod(j-2*k,n)] for j in range(n)])
    P = "ZZ"+"I"*(n-2)
    P2 = "".join([P[mod(j-2*k+1,n)] for j in range(n)])
    x_gate = get_gate(r_0, 'layer2', P1) +  get_gate(r_0, 'layer2', P2) 
    print(P1,P2,np.exp(-x_gate))
    ave_err += np.abs((1-2*eps) * (1-2*eps)**2 - np.exp(-x_gate))
ave_err/=n/2
print("average estimation error =",ave_err)

True 1q Gate Z fidelity (layer 2): 0.98
IZIIIIIIIIII 0.9848267377486163
IIIZIIIIIIII 0.981621400857668
IIIIIZIIIIII 0.973116089613035
IIIIIIIZIIII 0.9812398042414359
IIIIIIIIIZII 0.9745831638877591
IIIIIIIIIIIZ 0.9820408163265306
average estimation error = 0.0036715842789094566
Below are gauge-dependent (unlearnable) fidelity
ZIIIIIIIIIII 0.9617663801407972
IIZIIIIIIIII 0.969702341813577
IIIIZIIIIIII 0.9663287716241229
IIIIIIZIIIII 0.9713173454409495
IIIIIIIIZIII 0.972320939976874
IIIIIIIIIIZI 0.9604022654179509
average estimation error = 0.013026992597621412 (Large error due to unlearnability)
Learnable combined cycles (IZ->ZZ). True value = 0.9411919999999999
ZIIIIIIIIIII ZIIIIIIIIIIZ 0.9341385452510902
IIZIIIIIIIII IZZIIIIIIIII 0.9506748568244526
IIIIZIIIIIII IIIZZIIIIIII 0.9390572309544528
IIIIIIZIIIII IIIIIZZIIIII 0.9381355244131045
IIIIIIIIZIII IIIIIIIZZIII 0.9511106654028018
IIIIIIIIIIZI IIIIIIIIIZZI 0.9280026710255286
average estimation error = 0.007472591763846348
